# Amazon Personalize AWS User Personalization + Contextual Recommendations Example


## Introduction <a class="anchor" id="intro"></a>

For the most part, the algorithms in Amazon Personalize (called recipes) look to solve different tasks, explained here:

1. **User Personalization** - Recommends items based on previous user interactions with items.
1. **Personalized-Ranking** - Takes a collection of items and then orders them in probable order of interest using an HRNN-like approach.
1. **SIMS (Similar Items)** - Given one item, recommends other items also interacted with by users.

No matter the use case, the algorithms all share a base of learning on user-item-interaction data which is defined by 3 core attributes:

1. **UserID** - The user who interacted
1. **ItemID** - The item the user interacted with
1. **Timestamp** - The time at which the interaction occurred


## Choose a dataset or data source <a class="anchor" id="source"></a>
[Back to top](#top)

As we mentioned, the user-item-iteraction data is key for getting started with the service. This means we need to look for use cases that generate that kind of data, a few common examples are:

1. Video-on-demand applications
1. E-commerce platforms
1. Social media aggregators / platforms

There are a few guidelines for scoping a problem suitable for Personalize. We recommend the values below as a starting point, although the [official limits](https://docs.aws.amazon.com/personalize/latest/dg/limits.html) lie a little lower.

* Authenticated users
* At least 50 unique users
* At least 100 unique items
* At least 2 dozen interactions for each user 

Most of the time this is easily attainable, and if you are low in one category, you can often make up for it by having a larger number in another category.

Generally speaking your data will not arrive in a perfect form for Personalize, and will take some modification to be structured correctly. This notebook looks to guide you through all of that. 

To begin with, we are going to use an airlines review dataset. A scraped dataset created from all user reviews found on Skytrax (www.airlinequality.com). The data can be found at https://github.com/quankiquanki/skytrax-reviews-dataset 

In [1]:
import pandas as pd, numpy as np
import io
import scipy.sparse as ss
import json
import time
import datetime
import os
import sagemaker.amazon.common as smac
import boto3
import uuid
from botocore.exceptions import ClientError

sagemaker.config INFO - Not applying SDK defaults from location: /opt/homebrew/share/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/xian.zhang/Library/Application Support/sagemaker/config.yaml


### Import and Explore your dataset

In [2]:
data_dir = "airlines_data"
!mkdir $data_dir
!cd $data_dir && wget https://raw.githubusercontent.com/quankiquanki/skytrax-reviews-dataset/master/data/airline.csv


mkdir: airlines_data: File exists
--2025-05-30 12:16:44--  https://raw.githubusercontent.com/quankiquanki/skytrax-reviews-dataset/master/data/airline.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34752262 (33M) [text/plain]
Saving to: ‘airline.csv.1’

airline.csv.1       100%[===================>]  33.14M  4.85MB/s    in 6.9s    

2025-05-30 12:16:53 (4.77 MB/s) - ‘airline.csv.1’ saved [34752262/34752262]



In [3]:
airline_df = pd.read_csv(data_dir + '/airline.csv')
airline_df.head()

,airline_name,link,title,author,author_country,date,content,aircraft,type_traveller,cabin_flown,route,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,wifi_connectivity_rating,value_money_rating,recommended
0,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,D Ito,Germany,2015-04-10,Outbound flight FRA/PRN A319. 2 hours 10 min f...,NaN,NaN,Economy,NaN,7.0,4.0,4.0,4.0,0.0,NaN,NaN,4.0,1
1,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Ron Kuhlmann,United States,2015-01-05,Two short hops ZRH-LJU and LJU-VIE. Very fast ...,NaN,NaN,Business Class,NaN,10.0,4.0,5.0,4.0,1.0,NaN,NaN,5.0,1
2,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,E Albin,Switzerland,2014-09-14,Flew Zurich-Ljubljana on JP365 newish CRJ900. ...,NaN,NaN,Economy,NaN,9.0,5.0,5.0,4.0,0.0,NaN,NaN,5.0,1
3,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Tercon Bojan,Singapore,2014-09-06,Adria serves this 100 min flight from Ljubljan...,NaN,NaN,Business Class,NaN,8.0,4.0,4.0,3.0,1.0,NaN,NaN,4.0,1
4,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,L James,Poland,2014-06-16,WAW-SKJ Economy. No free snacks or drinks on t...,NaN,NaN,Economy,NaN,4.0,4.0,2.0,1.0,2.0,NaN,NaN,2.0,0


As we can see here the dataset has a lot of columns we can use to create the required data sets in Amazon Personalize.

The first thing we are going to do is make 2 copies of the dataset

In [4]:
a_interactions_df = airline_df.copy()
a_users_df = airline_df.copy()

## Building the Interactions Data set

Let's build the interactions dataset. By following the these steps:

- Drop the columns we are not interested in
- Create a new column to account for Event Type
- Rename the columns to a more standard naming convention for you Amazon Personalize import job



In [5]:
# Keeping only 5 columns
a_interactions_df = a_interactions_df[['airline_name', 'author', 'date', 'cabin_flown', 'overall_rating']]
# Creating an additional column for Event Type
a_interactions_df['EVENT_TYPE']='RATING'
# Making sure the author name is unique without spaces
a_interactions_df['author'] = a_interactions_df['author'].str.replace(" ","")
# Rename the columns to a more Amazon Personalize standar notation
a_interactions_df.rename(columns = {'airline_name':'ITEM_ID', 'author':'USER_ID',
                              'date':'TIMESTAMP', 'cabin_flown': 'CABIN_TYPE', 'overall_rating': 'EVENT_VALUE'}, inplace = True) 
a_interactions_df.head()

,ITEM_ID,USER_ID,TIMESTAMP,CABIN_TYPE,EVENT_VALUE,EVENT_TYPE
0,adria-airways,DIto,2015-04-10,Economy,7.0,RATING
1,adria-airways,RonKuhlmann,2015-01-05,Business Class,10.0,RATING
2,adria-airways,EAlbin,2014-09-14,Economy,9.0,RATING
3,adria-airways,TerconBojan,2014-09-06,Business Class,8.0,RATING
4,adria-airways,LJames,2014-06-16,Economy,4.0,RATING


Amazon Personalize supports **contextual recommendations**, through which you can improve relevance of recommendations by generating them within a context, for instance device type, location, time of day, etc. Contextual information is also useful in personalization for new/unidentified users even when the past interactions of these users are not known.

In our case we are going to use **Cabin Type** as a context to recommend which airline is the best fit for our user. Let's explore which values we are going to be able to pass as our context when getting recommendations


In [6]:
a_interactions_df.CABIN_TYPE.unique()

array(['Economy', 'Business Class', nan, 'Premium Economy', 'First Class'],
      dtype=object)

As we can see our current **Timestamp** value in the dataset is a string. Amazon Personalize requires the timestamp value as Unix type. Let's take a random timestamp value and convert it to Unix type

In [7]:
# Get a random value from the timestamp column
arb_time_stamp = a_interactions_df.iloc[50]['TIMESTAMP']
# Transform this string to date time
date_time_obj = datetime.datetime.strptime(arb_time_stamp, '%Y-%m-%d')
print('Date:', date_time_obj.date())
# Get the date of this object
d = date_time_obj.date()
# Transform the date object to Unix time
unixtime = time.mktime(d.timetuple())
print('Unix Time: ', unixtime)

Date: 2004-03-18
Unix Time:  1079596800.0


Now we are going to do the same transformation to all of our values in the timestamp column

In [8]:
# Define a function
def convert_to_unix(string_date):
    date_time_obj = datetime.datetime.strptime(string_date, '%Y-%m-%d')
    d = date_time_obj.date()
    return time.mktime(d.timetuple())

# Apply this function across the Timestamp column
a_interactions_df['TIMESTAMP'] = a_interactions_df['TIMESTAMP'].apply(convert_to_unix)
a_interactions_df.head(5)

,ITEM_ID,USER_ID,TIMESTAMP,CABIN_TYPE,EVENT_VALUE,EVENT_TYPE
0,adria-airways,DIto,1.428649e+09,Economy,7.0,RATING
1,adria-airways,RonKuhlmann,1.420445e+09,Business Class,10.0,RATING
2,adria-airways,EAlbin,1.410678e+09,Economy,9.0,RATING
3,adria-airways,TerconBojan,1.409987e+09,Business Class,8.0,RATING
4,adria-airways,LJames,1.402902e+09,Economy,4.0,RATING


Let's take a look at some of our dataset properties

In [9]:
a_interactions_df.describe()

,TIMESTAMP,EVENT_VALUE
count,4.139600e+04,36861.000000
mean,1.373976e+09,6.039527
std,5.771903e+07,3.214680
min,2.880000e+04,1.000000
25%,1.350889e+09,3.000000
50%,1.389686e+09,7.000000
75%,1.412147e+09,9.000000
max,1.438499e+09,10.000000


Are there any Null values??

In [10]:
a_interactions_df.isnull().values.any()

True

Let's drop those Null values and make sure there aren't any

In [11]:
a_interactions_df = a_interactions_df.dropna()
a_interactions_df.isnull().values.any()

False

Now that we have our data ready for Amazon Personalize, let's save it into a file locally

In [12]:
interactions_filename = "a_interactions.csv"
a_interactions_df.to_csv((data_dir + "/"+interactions_filename), index=False, float_format='%.0f')

## Building the Users Data set

Let's build the users dataset. By following the these steps:

- Drop the columns we are not interested in
- Create a new column to account for Nationality as user metadata
- Rename the columns to a more standard naming convention for you Amazon Personalize import job


In [13]:
# Copy the complete airlines data set
a_users_df = airline_df.copy()
# Select only interested columns
a_users_df = a_users_df[['author', 'author_country']]
# Clean up the authors string
a_users_df['author'] = a_users_df['author'].str.replace(" ","")
# Rename the columns
a_users_df.rename(columns = { 'author':'USER_ID', 'author_country':'NATIONALITY'}, inplace = True) 
# Drop any null values
a_users_df = a_users_df.dropna()
# Save your file locally
users_filename = "a_users.csv"
a_users_df.to_csv((data_dir +"/"+users_filename), index=False)

## Configure an S3 bucket and an IAM  role <a class="anchor" id="bucket_role"></a>
[Back to top](#top)

So far, we have downloaded, manipulated, and saved the data onto the Amazon EBS instance attached to instance running this Jupyter notebook. However, Amazon Personalize will need an S3 bucket to act as the source of your data, as well as IAM roles for accessing that bucket. Let's set all of that up.

Use the metadata stored on the instance underlying this Amazon SageMaker notebook, to determine the region it is operating in. If you are using a Jupyter notebook outside of Amazon SageMaker, simply define the region as a string below. The Amazon S3 bucket needs to be in the same region as the Amazon Personalize resources we have been creating so far.

In [15]:
# with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
#     data = json.load(notebook_info)
#     resource_arn = data['ResourceArn']
#     region = resource_arn.split(':')[3]
region = "us-east-2"

print(region)

us-east-2


In [17]:
# os.environ['AWS_DEFAULT_REGION'] = 'us-east-2'  # or your preferred region

# Configure the SDK to Personalize:
# personalize = boto3.client('personalize')
# personalize_runtime = boto3.client('personalize-runtime')

# Set your region
os.environ['AWS_DEFAULT_REGION'] = 'us-east-2'  # This matches the region you specified in SSO config

# Create a session with your SSO profile
session = boto3.Session(profile_name='767398024897_engineering')

# Create your clients using the session
personalize = session.client('personalize')
personalize_runtime = session.client('personalize-runtime')


Amazon S3 bucket names are globally unique. To create a unique bucket name, the code below will append the string `personalizepoc` to your AWS account number. Then it creates a bucket with this name in the region discovered in the previous cell.

In [18]:
s3 = session.client('s3')
account_id = session.client('sts').get_caller_identity().get('Account')
suffix = str(np.random.uniform())[4:9]
bucket_name = "personalize-user-personalization-example" + suffix
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

personalize-user-personalization-example00672


### Upload data to S3

Now that your Amazon S3 bucket has been created, upload the CSV file of our user-item-interaction data. 

In [19]:
interactions_filename = data_dir + '/a_interactions.csv'
session.resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)

In [20]:
user_metadata_file = data_dir + '/a_users.csv'
session.resource('s3').Bucket(bucket_name).Object(user_metadata_file).upload_file(user_metadata_file)

## Create dataset groups and the interactions dataset <a class="anchor" id="group_dataset"></a>
[Back to top](#top)

The highest level of isolation and abstraction with Amazon Personalize is a *dataset group*. Information stored within one of these dataset groups has no impact on any other dataset group or models created from one - they are completely isolated. This allows you to run many experiments and is part of how we keep your models private and fully trained only on your data. 

Before importing the data prepared earlier, there needs to be a dataset group and a dataset added to it that handles the interactions.

Dataset groups can house the following types of information:

* User-item-interactions
* Event streams (real-time interactions)
* User metadata
* Item metadata

Before we create the dataset group and the dataset for our interaction data, let's validate that your environment can communicate successfully with Amazon Personalize.

In [21]:
personalize = session.client(service_name='personalize')
personalize_runtime = session.client(service_name='personalize-runtime')
personalize_events = session.client(service_name='personalize-events')

### Create a Dataset Group

The following cell will create a new dataset group with the name *airlines-dataset-group* + a suffix

In [22]:
dataset_group_name = "airlines-dataset-group-" + suffix

create_dataset_group_response = personalize.create_dataset_group(
    name = dataset_group_name
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-2:767398024897:dataset-group/airlines-dataset-group-00672",
  "ResponseMetadata": {
    "RequestId": "59620f8e-3695-43fd-b988-9b75eacfb249",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 30 May 2025 19:25:29 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "107",
      "connection": "keep-alive",
      "x-amzn-requestid": "59620f8e-3695-43fd-b988-9b75eacfb249",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


Before we can use the dataset group, it must be active. This can take a minute or two. Execute the cell below and wait for it to show the ACTIVE status. It checks the status of the dataset group every second, up to a maximum of 3 hours.

In [23]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


Now that you have a dataset group, you can create a dataset for the interaction data.

# Create datasets

### Interactions Dataset

First, define a schema to tell Amazon Personalize what type of dataset you are uploading. There are several reserved and mandatory keywords required in the schema, based on the type of dataset. More detailed information can be found in the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Here, you will create a schema for interactions data, which needs the `USER_ID`, `ITEM_ID`, `TIMESTAMP`, `CABIN_TYPE`, `EVENT_TYPE`, `EVENT_VALUE`, and `TIMESTAMP` fields. These must be defined in the same order in the schema as they appear in the dataset.

In [24]:
schema_name="airlines-interaction-schema-"+suffix

In [25]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name":"CABIN_TYPE",
            "type": "string",
            "categorical": True
        },
        {
          "name": "EVENT_TYPE",
          "type": "string"
        },
        {
          "name": "EVENT_VALUE",
          "type": "float"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = schema_name,
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-2:767398024897:schema/airlines-interaction-schema-00672",
  "ResponseMetadata": {
    "RequestId": "37a6b629-4766-4b3b-86cb-16ab0f233a74",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 30 May 2025 19:25:49 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "99",
      "connection": "keep-alive",
      "x-amzn-requestid": "37a6b629-4766-4b3b-86cb-16ab0f233a74",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


In [26]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn,
    name = "airlines-dataset-interactions-" + suffix
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-2:767398024897:dataset/airlines-dataset-group-00672/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "1d8539da-fb3a-4c19-9787-7f783a92ea58",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 30 May 2025 19:25:50 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "109",
      "connection": "keep-alive",
      "x-amzn-requestid": "1d8539da-fb3a-4c19-9787-7f783a92ea58",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


### Users Dataset

Here, you will create a schema for the users data, which needs the `USER_ID`, and `NATIONALITY` fields. These must be defined in the same order in the schema as they appear in the dataset.


In [27]:
metadata_schema_name="airlines-users-schema-"+suffix

In [28]:
metadata_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "NATIONALITY",
            "type": "string",
            "categorical": True
        }
    ],
    "version": "1.0"
}

create_metadata_schema_response = personalize.create_schema(
    name = metadata_schema_name,
    schema = json.dumps(metadata_schema)
)

metadata_schema_arn = create_metadata_schema_response['schemaArn']
print(json.dumps(create_metadata_schema_response, indent=2))


{
  "schemaArn": "arn:aws:personalize:us-east-2:767398024897:schema/airlines-users-schema-00672",
  "ResponseMetadata": {
    "RequestId": "da9ac339-13db-49b3-ba1d-6248a02ad6c4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 30 May 2025 19:25:50 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "93",
      "connection": "keep-alive",
      "x-amzn-requestid": "da9ac339-13db-49b3-ba1d-6248a02ad6c4",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


In [29]:
dataset_type = "USERS"
create_metadata_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = metadata_schema_arn,
    name = "airlines-metadata-dataset-users-" + suffix
)

metadata_dataset_arn = create_metadata_dataset_response['datasetArn']
print(json.dumps(create_metadata_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-2:767398024897:dataset/airlines-dataset-group-00672/USERS",
  "ResponseMetadata": {
    "RequestId": "465474e3-8d70-42cd-b790-72b7415a78fa",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 30 May 2025 19:25:50 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "102",
      "connection": "keep-alive",
      "x-amzn-requestid": "465474e3-8d70-42cd-b790-72b7415a78fa",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


## Configure an S3 bucket and an IAM  role <a class="anchor" id="bucket_role"></a>

So far, we have downloaded, manipulated, and saved the data onto the Amazon EBS instance attached to instance running this Jupyter notebook. However, Amazon Personalize will need an S3 bucket to act as the source of your data, as well as IAM roles for accessing that bucket. Let's set all of that up.

Use the metadata stored on the instance underlying this Amazon SageMaker notebook, to determine the region it is operating in. If you are using a Jupyter notebook outside of Amazon SageMaker, simply define the region as a string below. The Amazon S3 bucket needs to be in the same region as the Amazon Personalize resources we have been creating so far.

### Set the S3 bucket policy
Amazon Personalize needs to be able to read the contents of your S3 bucket. So add a bucket policy which allows that.

In [31]:
s3 = session.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy));

### Create an IAM role

Amazon Personalize needs the ability to assume roles in AWS in order to have the permissions to execute certain tasks. Let's create an IAM role and attach the required policies to it. The code below attaches very permissive policies; please use more restrictive policies for any production application.

In [32]:
iam = session.client("iam")

role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise
        
# sometimes need to wait a bit for the role to be created
time.sleep(45)
print(role_arn)

arn:aws:iam::767398024897:role/PersonalizeS3Role-00672


# Create your Dataset import jobs

## Import the interactions data <a class="anchor" id="import"></a>

Earlier you created the dataset group and dataset to house your information, so now you will execute an import job that will load the data from the S3 bucket into the Amazon Personalize dataset. 

In [33]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "airlines-dataset-import-job-"+suffix,
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-2:767398024897:dataset-import-job/airlines-dataset-import-job-00672",
  "ResponseMetadata": {
    "RequestId": "173b7d43-b745-414a-bc08-332b5512339d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 30 May 2025 19:29:22 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "121",
      "connection": "keep-alive",
      "x-amzn-requestid": "173b7d43-b745-414a-bc08-332b5512339d",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


## Import the Users data <a class="anchor" id="import"></a>

Earlier you created the dataset group and dataset to house your information, so now you will execute an import job that will load the data from the S3 bucket into the Amazon Personalize dataset. 

In [34]:
create_metadata_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "airlines-users-metadata-dataset-import-job-"+suffix,
    datasetArn = metadata_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, user_metadata_file)
    },
    roleArn = role_arn
)

metadata_dataset_import_job_arn = create_metadata_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_metadata_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-2:767398024897:dataset-import-job/airlines-users-metadata-dataset-import-job-00672",
  "ResponseMetadata": {
    "RequestId": "5944c150-702d-4786-ab13-ad50bac3ccc5",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 30 May 2025 19:29:23 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "136",
      "connection": "keep-alive",
      "x-amzn-requestid": "5944c150-702d-4786-ab13-ad50bac3ccc5",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


### Wait for the Dataset Import Jobs to have ACTIVE Status

Before we can use the dataset, the import job must be active. Execute the cell below and wait for it to show the ACTIVE status. It checks the status of the import job every second, up to a maximum of 3 hours.

Importing the data can take some time, depending on the size of the dataset. In this demo, the data import job has been pre executed.

In [35]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


In [36]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = metadata_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


When the dataset import is active, you are ready to start building models with the AWS User Personalization recipe.

## Create solutions <a class="anchor" id="solutions"></a>
[Back to top](#top)

In this notebook, we will create a solution with the following recipe:

1. aws-user-personalization


In Amazon Personalize, a specific variation of an algorithm is called a recipe. Different recipes are suitable for different situations. A trained model is called a solution, and each solution can have many versions that relate to a given volume of data when the model was trained.

To start, we will list all the recipes that are supported. This will allow you to select one and use that to build your model.

In [37]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed
arn:aws:personalize:::recipe/aws-ecomm-frequently-bought-together
arn:aws:personalize:::recipe/aws-ecomm-popular-items-by-purchases
arn:aws:personalize:::recipe/aws-ecomm-popular-items-by-views
arn:aws:personalize:::recipe/aws-ecomm-recommended-for-you
arn:aws:personalize:::recipe/aws-item-affinity
arn:aws:personalize:::recipe/aws-item-attribute-affinity
arn:aws:personalize:::recipe/aws-next-best-action
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-personalized-ranking-v2
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-similar-items


The output is just a JSON representation of all of the algorithms mentioned in the introduction.

Next we will select specific recipes and build models with them.

### AWS User Personalization

AWS User Personalization is one of the more advanced recommendation models that you can use and it allows for real-time updates of recommendations based on user behavior. It also tends to outperform other approaches, like collaborative filtering. This recipe takes the longest to train, so let's start with this recipe first.

For our use case, using the Airlines reviews data, we can use the AWS User Personalization to recommend airlines to a user based on the user's previous artist tagging behavior. Remember, we used the tagging data to represent positive interactions between a user and an artist.

First, select the recipe by finding the ARN in the list of recipes above.

In [38]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

#### Create the solution

First you create a solution using the recipe. Although you provide the dataset ARN in this step, the model is not yet trained. See this as an identifier instead of a trained model.

Note that we have HPO activated here. This is a good idea when 


In [39]:
create_solution_response = personalize.create_solution(
    name = "airlines-user-personalization-solution-HPO-"+suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn,
    performHPO=True
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-2:767398024897:solution/airlines-user-personalization-solution-HPO-00672",
  "ResponseMetadata": {
    "RequestId": "91500400-478c-4173-bee7-61e22e94c187",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 30 May 2025 19:34:25 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "118",
      "connection": "keep-alive",
      "x-amzn-requestid": "91500400-478c-4173-bee7-61e22e94c187",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


#### Create the solution version

Once you have a solution, you need to create a version in order to complete the model training. The training can take a while to complete, upwards of 25 minutes, and an average of 40 minutes for this recipe with our dataset. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create many models and deploy them quickly. So we will set up the while loop for all of the solutions further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

In [40]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-2:767398024897:solution/airlines-user-personalization-solution-HPO-00672/200167b7",
  "ResponseMetadata": {
    "RequestId": "717c17b0-e09c-450b-a3bc-e48e6123a27f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 30 May 2025 19:34:25 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "134",
      "connection": "keep-alive",
      "x-amzn-requestid": "717c17b0-e09c-450b-a3bc-e48e6123a27f",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


In [41]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: ACTIVE


## Evaluate solution versions <a class="anchor" id="eval"></a>
[Back to top](#top)

It should not take more than an hour to train all the solutions from this notebook. While training is in progress, we recommend taking the time to read up on the various algorithms (recipes) and their behavior in detail. This is also a good time to consider alternatives to how the data was fed into the system and what kind of results you expect to see.

When the solutions finish creating, the next step is to obtain the evaluation metrics. Personalize calculates these metrics based on a subset of the training data. The image below illustrates how Personalize splits the data. Given 10 users, with 10 interactions each (a circle represents an interaction), the interactions are ordered from oldest to newest based on the timestamp. Personalize uses all of the interaction data from 90% of the users (blue circles) to train the solution version, and the remaining 10% for evaluation. For each of the users in the remaining 10%, 90% of their interaction data (green circles) is used as input for the call to the trained model. The remaining 10% of their data (orange circle) is compared to the output produced by the model and used to calculate the evaluation metrics.

![personalize metrics](static/imgs/personalize_metrics.png)

We recommend reading [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) to understand the metrics, but we have also copied parts of the documentation below for convenience.

You need to understand the following terms regarding evaluation in Personalize:

* *Relevant recommendation* refers to a recommendation that matches a value in the testing data for the particular user.
* *Rank* refers to the position of a recommended item in the list of recommendations. Position 1 (the top of the list) is presumed to be the most relevant to the user.
* *Query* refers to the internal equivalent of a GetRecommendations call.

The metrics produced by Personalize are:
- **coverage**: The proportion of unique recommended items from all queries out of the total number of unique items in the training data (includes both the Items and Interactions datasets).
- **mean_reciprocal_rank_at_25**: The [mean of the reciprocal ranks](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) of the first relevant recommendation out of the top 25 recommendations over all queries. This metric is appropriate if you're interested in the single highest ranked recommendation.
- **normalized_discounted_cumulative_gain_at_K**: Discounted gain assumes that recommendations lower on a list of recommendations are less relevant than higher recommendations. Therefore, each recommendation is discounted (given a lower weight) by a factor dependent on its position. To produce the [cumulative discounted gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (DCG) at K, each relevant discounted recommendation in the top K recommendations is summed together. The normalized discounted cumulative gain (NDCG) is the DCG divided by the ideal DCG such that NDCG is between 0 - 1. (The ideal DCG is where the top K recommendations are sorted by relevance.) Amazon Personalize uses a weighting factor of 1/log(1 + position), where the top of the list is position 1. This metric rewards relevant items that appear near the top of the list, because the top of a list usually draws more attention.
- **precision_at_K**: The number of relevant recommendations out of the top K recommendations divided by K. This metric rewards precise recommendation of the relevant items.

Let's take a look at the evaluation metrics for each of the solutions produced in this notebook. *Please note, your results might differ from the results described in the text of this notebook, due to the quality of the LastFM dataset.* 

### AWS User Personalizatioin metrics

First, retrieve the evaluation metrics for the AWS User Personalization solution version.

In [42]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))


{
  "solutionVersionArn": "arn:aws:personalize:us-east-2:767398024897:solution/airlines-user-personalization-solution-HPO-00672/200167b7",
  "metrics": {
    "coverage": 0.4131,
    "mean_reciprocal_rank_at_25": 0.2391,
    "normalized_discounted_cumulative_gain_at_10": 0.3075,
    "normalized_discounted_cumulative_gain_at_25": 0.3572,
    "normalized_discounted_cumulative_gain_at_5": 0.2738,
    "precision_at_10": 0.0444,
    "precision_at_25": 0.0257,
    "precision_at_5": 0.0689
  },
  "ResponseMetadata": {
    "RequestId": "c8ca894a-8ede-40bf-9e9f-7c300b440be6",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 30 May 2025 19:54:32 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "432",
      "connection": "keep-alive",
      "x-amzn-requestid": "c8ca894a-8ede-40bf-9e9f-7c300b440be6",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "

# Create a campaign from the solution

## Create campaigns <a class="anchor" id="create"></a>

A campaign is a hosted solution version; an endpoint which you can query for recommendations. Pricing is set by estimating throughput capacity (requests from users for personalization per second). When deploying a campaign, you set a minimum throughput per second (TPS) value. This service, like many within AWS, will automatically scale based on demand, but if latency is critical, you may want to provision ahead for larger demand. For this POC and demo, all minimum throughput thresholds are set to 1. For more information, see the [pricing page](https://aws.amazon.com/personalize/pricing/).

Let's start deploying the campaigns.

### AWS User Personalization

Deploy a campaign for your AWS User Personalization solution version. It can take around 10 minutes to deploy a campaign. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create multiple campaigns. So we will set up the while loop for all of the campaigns further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

In [43]:
create_campaign_response = personalize.create_campaign(
    name = "airlines-metadata-campaign-"+suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 2,    
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-2:767398024897:campaign/airlines-metadata-campaign-00672",
  "ResponseMetadata": {
    "RequestId": "9db36111-bbaa-49b2-b7ab-edb33dcd28e7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 30 May 2025 19:54:32 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "102",
      "connection": "keep-alive",
      "x-amzn-requestid": "9db36111-bbaa-49b2-b7ab-edb33dcd28e7",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


In [44]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


### AWS User Personalization

AWS User Personalization is one of the more advanced algorithms provided by Amazon Personalize. It supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since the AWS User Personalization algorithm relies on having a sampling of users, let's load the data we need for that and select 3 random users.

In [45]:
users_df = pd.read_csv(data_dir + '/a_users.csv')
# Render some sample data
users_df.sample(5)

,USER_ID,NATIONALITY
5772,RossWilson,Australia
32933,MassimilianoDeCarli,Oman
38953,DWood,Canada
6942,MJenkins,United States
14380,RNash,Australia


Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [46]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    
#   Context Recommendations
    context_options = ['None','Economy', 'Business Class','Premium Economy', 'First Class']
    
    for context in context_options:
        # Get the recommendations
        if context=='none':
            get_recommendations_response = personalize_runtime.get_recommendations(
                campaignArn = campaign_arn,
                userId = str(user_id),
            )
        else:
            get_recommendations_response = personalize_runtime.get_recommendations(
                campaignArn = campaign_arn,
                userId = str(user_id),
                context = {
                  'CABIN_TYPE': context
                }
            )
        # Build a new dataframe of recommendations
        item_list = get_recommendations_response['itemList']
        recommendation_list = []
        for item in item_list:
            recommendation_list.append(item['itemId'])
    #     print(recommendation_list)
        new_rec_DF = pd.DataFrame(recommendation_list, columns = [context])
        # Add this dataframe to the old one
        recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [47]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

         USER_ID     NATIONALITY
35125  AndyLetts  United Kingdom


,None,Economy,Business Class,Premium Economy,First Class
0,virgin-atlantic-airways,ryanair,british-airways,virgin-atlantic-airways,american-airlines
1,british-airways,thomson-airways,virgin-atlantic-airways,thomson-airways,british-airways
2,thomson-airways,monarch-airlines,etihad-airways,british-airways,emirates
3,air-transat,thomas-cook-airlines,qatar-airways,eva-air,united-airlines
4,eva-air,virgin-atlantic-airways,emirates,monarch-airlines,cathay-pacific-airways
5,american-airlines,flybe,oman-air,thomas-cook-airlines,etihad-airways
6,monarch-airlines,jet2-com,eva-air,cathay-pacific-airways,china-southern-airlines
7,vietnam-airlines,easyjet,srilankan-airlines,united-airlines,delta-air-lines
8,thomas-cook-airlines,wizz-air,bmi-british-midland-international,air-transat,virgin-atlantic-airways
9,emirates,british-airways,finnair,vietnam-airlines,us-airways


In [48]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

      USER_ID NATIONALITY
18075   FWong   Hong Kong


,None,Economy,Business Class,Premium Economy,First Class
0,cathay-pacific-airways,cathay-pacific-airways,cathay-pacific-airways,cathay-pacific-airways,cathay-pacific-airways
1,qantas-airways,singapore-airlines,qatar-airways,british-airways,american-airlines
2,emirates,qantas-airways,emirates,qantas-airways,emirates
3,qatar-airways,emirates,qantas-airways,eva-air,united-airlines
4,thai-airways,qatar-airways,singapore-airlines,united-airlines,thai-airways
5,singapore-airlines,thai-airways,thai-airways,air-france,british-airways
6,british-airways,dragonair,malaysia-airlines,vietnam-airlines,singapore-airlines
7,eva-air,malaysia-airlines,british-airways,air-new-zealand,qantas-airways
8,united-airlines,korean-air,dragonair,turkish-airlines,swiss-international-air-lines
9,dragonair,hong-kong-airlines,swiss-international-air-lines,thai-airways,qatar-airways


Here we clearly see that the recommendations for each user are different. If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export, which will be covered later in this notebook.

The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in media workloads, like video-on-demand, where a customer's intent may differ based on if they are watching with their children or on their own.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

#### Real time events

Start by creating an event tracker that is attached to the campaign.

In [49]:
response = personalize.create_event_tracker(
    name='AirlinesEventsTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-2:767398024897:event-tracker/86211a91
aa755465-992e-467d-80f0-2f453d4b8247


We will create some code that simulates a user interacting with a particular item. After running this code, you will get recommendations that differ from the results above.

We start by creating some methods for the simulation of real time events.

In [50]:
session_dict = {}

def send_user_rating(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
        "itemId": str(ITEM_ID),
        "eventValue": 10,
        "cabinType": "Economy"
    }
    event_json = json.dumps(event)
        
    print(f"Sent event for user {USER_ID} and item {ITEM_ID} with trackingId {TRACKING_ID}")
    # Make Call
    personalize_events.put_events(
        trackingId = TRACKING_ID,
        userId= str(USER_ID),
        sessionId = session_ID,
        eventList = [{
            'sentAt': int(time.time()),
            'eventType': 'RATING',
            'properties': event_json
            }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id):
    # Interact with the airline
    # Sending a rating of 10 in Economy class for the airline with that user
    send_user_rating(USER_ID=user_id, ITEM_ID=item_id)
    
    
    #   Context Recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id),
        context = {
          'CABIN_TYPE': 'Economy'
        }
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        recommendation_list.append(item['itemId'])
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [item_id+'|Economy'])
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df


At this point, we haven't generated any real-time events yet; we have only set up the code. To compare the recommendations before and after the real-time events, let's pick one user and generate the original recommendations for them.

## Recommendations before using the Event Tracker

In [51]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)
user_id = users[0]
recommendations_df_users

        USER_ID     NATIONALITY
39716  PColeman  United Kingdom


,None,Economy,Business Class,Premium Economy,First Class
0,qatar-airways,qatar-airways,qatar-airways,british-airways,emirates
1,turkish-airlines,turkish-airlines,emirates,cathay-pacific-airways,lufthansa
2,emirates,emirates,turkish-airlines,klm-royal-dutch-airlines,american-airlines
3,lufthansa,lufthansa,etihad-airways,turkish-airlines,qatar-airways
4,british-airways,klm-royal-dutch-airlines,british-airways,air-france,cathay-pacific-airways
5,cathay-pacific-airways,etihad-airways,lufthansa,united-airlines,british-airways
6,klm-royal-dutch-airlines,alitalia,cathay-pacific-airways,lufthansa,united-airlines
7,american-airlines,ethiopian-airlines,klm-royal-dutch-airlines,virgin-atlantic-airways,etihad-airways
8,air-france,british-airways,malaysia-airlines,brussels-airlines,thai-airways
9,united-airlines,cathay-pacific-airways,avianca,vietnam-airlines,delta-air-lines


In [52]:
user_id

'PColeman'

Ok, so now we have a list of recommendations for this user before we have applied any real-time events. Now let's pick 3 random artists which we will simulate our user interacting with, and then see how this changes the recommendations.

In [53]:
# Next generate 3 random Airlines
airlines = a_interactions_df.sample(3)['ITEM_ID'].tolist()
airlines

['qantas-airways', 'icelandair', 'air-china']

In [54]:
user_recommendations_df = pd.DataFrame()
# Note this will take about 15 seconds to complete due to the sleeps
for airline in airlines:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, airline)
    time.sleep(5)
print(user_id)
user_recommendations_df

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 user_recommendations_df = pd.DataFrame()                                                     │
│   2 # Note this will take about 15 seconds to complete due to the sleeps                         │
│   3 for airline in airlines:                                                                     │
│ ❱ 4 │   user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommenda     │
│   5 │   time.sleep(5)                                                                            │
│   6 print(user_id)                                                                               │
│   7 user_recommendations_df                                                                      │
│                                                                                                  │
│ in get_new_recommendations_df_users_real_time:38                                                 │
│                                                                                                  │
│   35 def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id):       │
│   36 │   # Interact with the airline                                                             │
│   37 │   # Sending a rating of 10 in Economy class for the airline with that user                │
│ ❱ 38 │   send_user_rating(USER_ID=user_id, ITEM_ID=item_id)                                      │
│   39 │                                                                                           │
│   40 │                                                                                           │
│   41 │   #   Context Recommendations                                                             │
│                                                                                                  │
│ in send_user_rating:24                                                                           │
│                                                                                                  │
│   21 │   event_json = json.dumps(event)                                                          │
│   22 │                                                                                           │
│   23 │   # Make Call                                                                             │
│ ❱ 24 │   personalize_events.put_events(                                                          │
│   25 │   │   trackingId = TRACKING_ID,                                                           │
│   26 │   │   userId= str(USER_ID),                                                               │
│   27 │   │   sessionId = session_ID,                                                             │
│                                                                                                  │
│ /opt/homebrew/anaconda3/envs/personalize/lib/python3.9/site-packages/botocore/client.py:595 in   │
│ _api_call                                                                                        │
│                                                                                                  │
│    592 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    593 │   │   │   │   )                                                                         │
│    594 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  595 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    596 │   │                                                                                     │
│    597 │   │   _api_call.__name__ = str(py_operation_name) 

In the cell above, the first column after the index is the user's default recommendations from the AWS User Personalization model, and each column after that has a header of the airlines that they interacted with via a real time event, and the recommendations after this event occurred. 

The behavior may not shift very much; this is due to the relatively limited nature of this dataset. If you wanted to better understand this, try simulating rating random airlines with random ratings, and you should see a more pronounced impact.